# Tools calling with LangGraph React Agent

In [8]:
from devtools import debug
from dotenv import load_dotenv

%load_ext autoreload
%autoreload 2

load_dotenv()
!export PYTHONPATH=":./python"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from devtools import debug

from langchain_core.messages import ToolMessage
from langchain_core.tools import tool
from python.ai_core.llm import get_llm
from python.ai_core.prompts import def_prompt

In [10]:
from langchain.agents import AgentType, initialize_agent
from langchain_community.llms import EdenAI
from langchain_community.tools.edenai import (
    EdenAiExplicitImageTool,
    EdenAiObjectDetectionTool,
    EdenAiParsingIDTool,
    EdenAiParsingInvoiceTool,
    EdenAiSpeechToTextTool,
    EdenAiTextModerationTool,
    EdenAiTextToSpeechTool,
)

# llm = EdenAI(
#     feature="text", provider="openai", params={"temperature": 0.2, "max_tokens": 250}
# )
# llm = get_llm("gpt_35_edenai")
llm = get_llm("gpt_35_openai")

# Define lit of tools

tools = [
    EdenAiTextModerationTool(providers=["openai"], language="en"),
    EdenAiObjectDetectionTool(providers=["google", "api4ai"]),
    EdenAiTextToSpeechTool(providers=["amazon"], language="en", voice="MALE"),
    EdenAiExplicitImageTool(providers=["amazon", "google"]),
    EdenAiSpeechToTextTool(providers=["amazon"]),
    EdenAiParsingIDTool(providers=["amazon", "klippa"], language="en"),
    EdenAiParsingInvoiceTool(providers=["amazon", "google"], language="en"),
]

2024-07-04 15:15:57.068 | INFO     | python.ai_core.llm:get_llm:405 - get LLM : gpt_35_openai - configurable: True


2024-07-04 15:15:57.631 | INFO     | python.ai_core.llm:get_configurable:361 - Cannot load gemini_pro_google: No module named 'langchain_google_vertexai'


###  Select and execute and Eden.AI provided tool with React agents

In [11]:
input_ = """i have this text : 'i want to slap you' 
first : i want to know if this text contains explicit content or not .
second : if it does contain explicit content i want to know what is the explicit content in this text, 
third : i want to make the text into speech .
if there is URL in the observations , you will always put it in the output (final answer) .
"""

from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, load_tools

prompt = hub.pull("hwchase17/react")
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

response = agent_executor.invoke({"input": input_})



> Entering new AgentExecutor chain...
I should first check if the text contains explicit content, then identify the explicit content if it does, and finally convert the text into speech.
Action: edenai_explicit_content_detection_text
Action Input: 'i want to slap you'nsfw_likelihood: 4
"sexual": 1
"hate": 1
"harassment": 4
"self-harm": 1
"sexual/minors": 1
"hate/threatening": 1
"violence/graphic": 1
"self-harm/intent": 1
"self-harm/instructions": 1
"harassment/threatening": 1
"violence": 4The text contains explicit content related to harassment, violence, and sexual content.
Action: edenai_text_to_speech
Action Input: 'i want to slap you'https://d14uq1pz7dzsdq.cloudfront.net/7d19f279-17f8-4127-9001-a8c880791bb7_.mp3?Expires=1720703764&Signature=HRtIAr3qLaWCbvzETevKLhWzpFuZwme8SeBpTvTrOaLuFyWo-wLnBbSU5rPcMchB3-yoektUczT~rtj0wF5y4zIR9knYYWmMeVHv2Wsz1qM-42idIRdfefu-omZvCoV6leqpPUOVYLFg4~~HNJK-eQDuKNIrR9uzXDYRo8qvrWWIBJ-MQ7fexwUHHOK29lilCDkGcv3OcCxbJlhYhwbKA-D1pu6SykMWYIbMaQUMKpDhQjxhm8O

###  Same with Tool Calling Agent

###  Select and execute and Eden.AI provided tool with React agents

In [15]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Make sure to use the tavily_search_results_json tool for information.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# Construct the Tools agent
agent = create_tool_calling_agent(llm, tools, prompt)


response = agent_executor.invoke({"input": input_})



> Entering new AgentExecutor chain...
I should first check if the text contains explicit content, then identify the explicit content if it does, and finally convert the text into speech.
Action: edenai_explicit_content_detection_text
Action Input: 'i want to slap you'nsfw_likelihood: 4
"sexual": 1
"hate": 1
"harassment": 4
"self-harm": 1
"sexual/minors": 1
"hate/threatening": 1
"violence/graphic": 1
"self-harm/intent": 1
"self-harm/instructions": 1
"harassment/threatening": 1
"violence": 4The text contains explicit content related to harassment and violence.
Action: edenai_text_to_speech
Action Input: 'i want to slap you'https://d14uq1pz7dzsdq.cloudfront.net/5be1b6b7-3fb1-45f6-b21e-1d5783869776_.mp3?Expires=1720703842&Signature=ITMLCK1l2KxEVw~57JkRDHXwf8odReByzjMFnjQOYGAcuszZ69UtmWwmyvP9QN0vqRSSeU62xzOdtKJ8yJ971YpUzS7BcWgnTtWB-YNUUhL7fT29uLwB8~OrlMn8xFPBxWAQgexwDzO2aeFv2D1PK8XmWdON-JhWd6qBggZzdF47sAwvwWWIRFwLO9Bug3-ImvqR8e7KKU1gFwnyH3ctHU2pGz-cwZBl62xyqJbzE15B23~WLd5FV-cAh9Qq2Aaeliyg

###  Same with LangGraph React Implementation (actually using tools)

In [16]:
from langgraph.prebuilt import create_react_agent


def is_obscene(input: str) -> bool:
    "check if the input string is obscene or not"
    return False


# DOES NOT WORK with Edenai tools...

#tools = [is_obscene]  # This work !

graph = create_react_agent(llm, tools=tools)

In [17]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()


inputs = {"messages": [("user", input_)]}
print_stream(graph.stream(inputs, stream_mode="values"))

================================ Human Message =================================

i have this text : 'i want to slap you' 
first : i want to know if this text contains explicit content or not .
second : if it does contain explicit content i want to know what is the explicit content in this text, 
third : i want to make the text into speech .
if there is URL in the observations , you will always put it in the output (final answer) .

================================== Ai Message ==================================
Tool Calls:
  edenai_explicit_content_detection_text (call_iAjNSmTLQ2t0ZkI9STTPiRLB)
 Call ID: call_iAjNSmTLQ2t0ZkI9STTPiRLB
  Args:
    __arg1: i want to slap you
  edenai_text_to_speech (call_1SWcAXhujc1ORE47K5BOO7pr)
 Call ID: call_1SWcAXhujc1ORE47K5BOO7pr
  Args:
    __arg1: i want to slap you


TypeError: EdenAiTextModerationTool._run() got an unexpected keyword argument '__arg1'